#NOT WORKING STUFF



Installing dependencies

In [ ]:
!pip install tensorflow
!pip install scikit.learn
!pip install pandas
!pip install numpy

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

In [ ]:
def convert(imgf, labelf, outf, n):
    f = open(imgf, "rb")
    o = open(outf, "w")
    l = open(labelf, "rb")

    f.read(16)
    l.read(8)
    images = []

    for i in range(n):
        image = [ord(l.read(1))]
        for j in range(28*28):
            image.append(ord(f.read(1)))
        images.append(image)

    for image in images:
        o.write(",".join(str(pix) for pix in image)+"\n")
    f.close()
    o.close()
    l.close()

convert("train-images-idx3-ubyte", "train-labels-idx1-ubyte",
"mnist_train.csv", 60000)
convert("t10k-images-idx3-ubyte", "t10k-labels-idx1-ubyte",
"mnist_test.csv", 10000)

In [ ]:
df_orig_train = pd.read_csv('mnist_train.csv')
df_orig_test = pd.read_csv('mnist_test.csv')

In [ ]:
df_orig_train.rename(columns={'5':'label'}, inplace=True)
df_orig_test.rename(columns={'7':'label'}, inplace=True)

In [ ]:
df_orig_train = pd.read_csv('mnist_train_final.csv')
df_orig_test = pd.read_csv('mnist_test_final.csv')
df_orig_train.head

In [ ]:
labels_train = df_orig_train['label'].to_numpy()
df_digits = df_orig_train.drop('label',axis=1)
labels_test = df_orig_test['label'].to_numpy()
df_digits_test = df_orig_test.drop('label',axis=1)
# ind = 3
# digits_data_train = df_digits.to_numpy()
# digit = digits_data_train[ind]
# digit = digit.reshape(28,28)
# plt.title('this is  --->   ' + str(labels_train[ind]))
# plt.imshow(digit, cmap='gray')

In [ ]:
df_orig_train.to_csv('mnist_train_final.csv', index=False)
df_orig_test.to_csv('mnist_test_final.csv', index=False)

PCA testing

In [ ]:
dim_red = 4
df_digits=StandardScaler().fit_transform(df_digits)
pca = PCA(n_components=dim_red, svd_solver='randomized',
          whiten=True).fit(df_digits)
data_train_pca = pca.transform(df_digits)

In [ ]:
df_digits_test=StandardScaler().fit_transform(df_digits_test)
pca = PCA(n_components=dim_red, svd_solver='randomized',
          whiten=True).fit(df_digits_test)
data_test_pca = pca.transform(df_digits_test)

In [ ]:
# param_grid = { "C" : [2]
#               , "gamma" : [2]}
# rf = svm.SVC()
# gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=2, n_jobs=-1, verbose=1)
# gs = gs.fit(data_train_pca, labels_train)

# print(gs.best_score_)
# print(gs.best_params_)

In [ ]:
# bp = gs.best_params_
# clf = svm.SVC(C=bp['C'], kernel='rbf', gamma=bp['gamma'])
clf = svm.SVC().fit(data_train_pca, labels_train)


In [ ]:
clf.score(data_test_pca,labels_test)

## My code 

Importing Dataset

In [ ]:
ds = tfds.load('mnist', split='train', as_supervised=True)
dstest = tfds.load('mnist', split='test', as_supervised=True)

In [ ]:
dsinfo=tfds.builder('mnist').info
mnist_frame_train=tfds.as_dataframe(ds,dsinfo)
mnist_frame_test=tfds.as_dataframe(dstest,dsinfo)

In [ ]:
data_train = mnist_frame_train.drop(['label'], axis='columns', inplace=False)
label_train = mnist_frame_train['label']
data_test= mnist_frame_test.drop(['label'], axis='columns', inplace=False)
label_test = mnist_frame_test['label']
data_train.image.shape

PCA self

In [ ]:
size_data=60000
dim_number=28*28
reduced_dim=16

dataarraymnist=np.zeros((size_data,dim_number))
labelarraymnist=np.zeros((size_data,1))
count=0

for i in mnist_frame_train.image:
  dataarraymnist[count][:]=np.reshape(i,(1,28*28))
  count=count+1

counttemp=0
for i in mnist_frame_train.label:
  labelarraymnist[counttemp][:]=i
  counttemp+=1

for i in range(len(labelarraymnist)):
  labelarraymnist[i][0]=int(labelarraymnist[i][0])

labelarraymnist=labelarraymnist.flatten()

dataPCA=StandardScaler().fit_transform(dataarraymnist)

pca=PCA(n_components=reduced_dim)

dataarraymnistPCA = pca.fit_transform(dataPCA)

print(dataarraymnistPCA.shape)
print(labelarraymnist.shape)

In [ ]:
cl=svm.SVC(kernel='rbf',gamma='scale').fit(dataarraymnistPCA,labelarraymnist)

In [ ]:
size_test=10000
dataarraymnisttest=np.zeros((size_test,dim_number))
labelarraymnisttest=np.zeros((size_test,1))

count=0
for i in mnist_frame_test.image:
  dataarraymnisttest[count][:]=np.reshape(i,(1,28*28))
  count=count+1

counttemp=0
for i in mnist_frame_test.label:
  labelarraymnisttest[counttemp][:]=i
  counttemp+=1

for i in range(len(labelarraymnisttest)):
  labelarraymnisttest[i][0]=int(labelarraymnisttest[i][0])

labelarraymnisttest=labelarraymnisttest.flatten()

dataPCAtest=StandardScaler().fit_transform(dataarraymnisttest)

pca=PCA(n_components=reduced_dim)

dataarraymnistPCAtest = pca.fit_transform(dataPCAtest)

print(dataarraymnistPCAtest.shape)

In [ ]:
predictedlabels=cl.predict(dataarraymnistPCAtest)

In [ ]:
count=0;

for i in range(len(predictedlabels)):
  if predictedlabels[i]!=labelarraymnisttest[i]:
    count+=1

print(count*100/len(predictedlabels))

print(predictedlabels)
print(labelarraymnisttest)

#TRYING AGAIN

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [ ]:
data_train=pd.read_csv('mnist_train.csv',header=None)
data_test=pd.read_csv('mnist_test.csv',header=None)

In [ ]:
size_train=60000
size_test=10000

In [ ]:
label_train=data_train.iloc[:, [0]]
data_train=data_train.drop(data_train.columns[0],axis=1)
label_test=data_test.iloc[:, [0]]
data_test=data_test.drop(data_train.columns[0],axis=1)
# label_train
# data_train

In [ ]:
data_train=data_train.to_numpy()
label_train=label_train.to_numpy()
data_test=data_test.to_numpy()
label_test=label_test.to_numpy()

PCA

In [ ]:
dim_redn = 4
pca = PCA(n_components=dim_redn, svd_solver='randomized',
          whiten=True).fit(data_train)
data_train_pca = pca.transform(data_train)
pca = PCA(n_components=dim_redn, svd_solver='randomized',
          whiten=True).fit(data_test)
data_test_pca = pca.transform(data_test)

SVM

In [ ]:
label_train=label_train.flatten()
modelSVM=svm.SVC(gamma='scale').fit(data_train,label_train)
modelSVM.score(data_test,label_test)

#AGANE


### SVM and Preprocessing

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from keras.datasets import mnist
from sklearn.preprocessing import scale
from sklearn.model_selection import GridSearchCV

In [ ]:
(data_train,label_train),(data_test,label_test)=mnist.load_data()
data_train_reshaped= data_train.reshape(data_train.shape[0], -1)
data_test_reshaped= data_test.reshape(data_test.shape[0], -1)

Error: Session cannot generate requests

In [4]:
data_complete=np.concatenate((data_train_reshaped,data_test_reshaped))
data_complete=data_complete/255.0
data_complete=scale(data_complete)
data_complete=data_complete.tolist()

Linear Kernel

Without Dimn Redn

In [ ]:
model_svm_linear=SVC(kernel='linear').fit(data_train_reshaped,label_train)
model_svm_linear.score(data_test_reshaped,label_test)

In [ ]:
model_svm_linear.score(data_train_reshaped,label_train )

PCA

In [ ]:
dim_redn=331 #Maybe try to convert to variance I want estimated
pca=PCA(n_components=dim_redn)
data_PCA_complete=pca.fit_transform(data_complete)

In [ ]:
data_PCA_train=data_PCA_complete[:60000][:]
data_PCA_test=data_PCA_complete[60000:][:]

In [ ]:
data_PCA_complete.shape
data_PCA_train.shape
data_PCA_test.shape

With PCA

In [ ]:
model_svm_linear=SVC(kernel='linear').fit(data_PCA_train,label_train)
model_svm_linear.score(data_PCA_test,label_test)

Possibly wrong LDA since didnt consider whole dataset

In [ ]:
dim_redn_LDA=9
lda=LDA(n_components=dim_redn_LDA)
data_LDA_train=lda.fit_transform(data_train_reshaped,label_train)
data_LDA_test=lda.fit_transform(data_test_reshaped,label_test)

With LDA

In [ ]:
model_svm_linear=SVC(kernel='linear').fit(data_LDA_train,label_train)
model_svm_linear.score(data_LDA_test,label_test)

RBF Kernel

With PCA

In [ ]:
#Searching for optimal parameters

# param_grid = { "C" : [0.1]
#               , "gamma" : [0.1]}
# rf = SVC()
# gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=2, n_jobs=-1, verbose=1)
# gs = gs.fit(data_PCA_train, label_train)
# bp = gs.best_params_

In [ ]:
# model_svm_linear=SVC(C=bp['C'], kernel='rbf', gamma=bp['gamma']).fit(data_PCA_train,label_train)
model_svm_linear=SVC( kernel='rbf' ).fit(data_PCA_train,label_train)
model_svm_linear.score(data_PCA_test,label_test)

Polynomical Kernel

With PCA

In [ ]:
model_svm_linear=SVC(kernel='poly').fit(data_PCA_train,label_train)
model_svm_linear.score(data_PCA_test,label_test)

## CNN Trying


In [4]:
paddings=tf.constant([[0,0],[2,2],[2,2]])
data_train_padded=tf.pad(data_train,paddings,"CONSTANT")
data_train_padded.shape
data_test_padded=tf.pad(data_test,paddings,"CONSTANT")
data_test_padded.shape

2021-11-25 20:25:44.635820: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorShape([10000, 32, 32])

In [5]:
data_train_padded=data_train_padded/255
data_test_padded=data_test_padded/255

In [6]:
data_train_padded_array=np.array(data_train_padded)/255.0
data_test_padded_array=np.array(data_test_padded)/255.0

In [7]:
data_train_CNN= data_train_padded_array.reshape(data_train_padded_array.shape[0], -1)
data_test_CNN= data_test_padded_array.reshape(data_test_padded_array.shape[0], -1)
data_train_CNN.shape

(60000, 1024)

In [8]:
data_train_CNN_array=data_train_CNN/255.0
data_test_CNN_array=data_test_CNN/255.0

In [9]:
[x,y,z]=data_train_padded.shape

In [10]:
#Adding Channel information (Grayscale)
data_train_CNN = tf.expand_dims(data_train_padded, axis=3, name=None)
data_test_CNN = tf.expand_dims(data_test_padded, axis=3, name=None)

In [11]:
from tensorflow.keras import layers,losses,models

In [84]:
#Defining LeNet5

model = models.Sequential()
model.add(layers.Conv2D(6, 5, activation='tanh', input_shape=data_train_CNN.shape[1:]))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))
model.add(layers.Conv2D(16, 5, activation='tanh'))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))
model.add(layers.Conv2D(120, 5, activation='tanh'))
model.add(layers.Flatten())
model.add(layers.Dense(84, activation='tanh'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d_4 (Averag  (None, 14, 14, 6)        0         
 ePooling2D)                                                     
                                                                 
 activation_4 (Activation)   (None, 14, 14, 6)         0         
                                                                 
 conv2d_12 (Conv2D)          (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_5 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 activation_5 (Activation)   (None, 5, 5, 16)        

In [85]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = model.fit(data_train_CNN, label_train, batch_size=64, epochs=40, validation_data=(data_test_CNN, label_test))

Epoch 1/40
938/938 [==============================] - 12s 12ms/step - loss: 1.3438 - accuracy: 0.5166 - val_loss: 0.5144 - val_accuracy: 0.8367
Epoch 2/40
938/938 [==============================] - 12s 13ms/step - loss: 0.3603 - accuracy: 0.8882 - val_loss: 0.3156 - val_accuracy: 0.9043
Epoch 3/40
938/938 [==============================] - 12s 13ms/step - loss: 0.2658 - accuracy: 0.9160 - val_loss: 0.2471 - val_accuracy: 0.9235
Epoch 4/40
938/938 [==============================] - 12s 13ms/step - loss: 0.2156 - accuracy: 0.9316 - val_loss: 0.1704 - val_accuracy: 0.9459
Epoch 5/40
938/938 [==============================] - 12s 13ms/step - loss: 0.1836 - accuracy: 0.9409 - val_loss: 0.1568 - val_accuracy: 0.9510
Epoch 6/40
938/938 [==============================] - 12s 12ms/step - loss: 0.1608 - accuracy: 0.9497 - val_loss: 0.1203 - val_accuracy: 0.9627
Epoch 7/40
938/938 [==============================] - 12s 13ms/step - loss: 0.1397 - accuracy: 0.9565 - val_loss: 0.1193 - val_accuracy:

In [86]:
model.evaluate(data_test_CNN, label_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0525 - accuracy: 0.9829


[0.05252949893474579, 0.9829000234603882]

Same Architecture changing activation functions

In [87]:
model = models.Sequential()
model.add(layers.Conv2D(6, 5, activation='relu', input_shape=data_train_CNN.shape[1:]))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(16, 5, activation='relu'))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(120, 5, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(84, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d_6 (Averag  (None, 14, 14, 6)        0         
 ePooling2D)                                                     
                                                                 
 activation_6 (Activation)   (None, 14, 14, 6)         0         
                                                                 
 conv2d_15 (Conv2D)          (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_7 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 activation_7 (Activation)   (None, 5, 5, 16)        

In [88]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = model.fit(data_train_CNN, label_train, batch_size=64, epochs=40, validation_data=(data_test_CNN, label_test))

Epoch 1/40
938/938 [==============================] - 13s 13ms/step - loss: 0.2815 - accuracy: 0.9160 - val_loss: 0.1144 - val_accuracy: 0.9641
Epoch 2/40
938/938 [==============================] - 12s 13ms/step - loss: 0.0868 - accuracy: 0.9728 - val_loss: 0.0598 - val_accuracy: 0.9800
Epoch 3/40
938/938 [==============================] - 11s 12ms/step - loss: 0.0612 - accuracy: 0.9806 - val_loss: 0.0431 - val_accuracy: 0.9869
Epoch 4/40
938/938 [==============================] - 13s 14ms/step - loss: 0.0472 - accuracy: 0.9853 - val_loss: 0.0489 - val_accuracy: 0.9826
Epoch 5/40
938/938 [==============================] - 14s 15ms/step - loss: 0.0393 - accuracy: 0.9873 - val_loss: 0.0412 - val_accuracy: 0.9866
Epoch 6/40
938/938 [==============================] - 13s 14ms/step - loss: 0.0332 - accuracy: 0.9895 - val_loss: 0.0464 - val_accuracy: 0.9841
Epoch 7/40
938/938 [==============================] - 12s 13ms/step - loss: 0.0298 - accuracy: 0.9906 - val_loss: 0.0361 - val_accuracy:

In [89]:
model.evaluate(data_test_CNN, label_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0550 - accuracy: 0.9896


[0.05504418909549713, 0.9896000027656555]

Changing Kernel Size

In [104]:
model = models.Sequential()
model.add(layers.Conv2D(6, 3, activation='relu', input_shape=data_train_CNN.shape[1:]))
model.add(layers.Conv2D(6, 3, activation='relu', input_shape=data_train_CNN.shape[1:]))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(16, 3, activation='relu'))
model.add(layers.Conv2D(16, 3, activation='relu'))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(120, 3, activation='relu'))
model.add(layers.Conv2D(120, 3, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(84, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()


Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_64 (Conv2D)          (None, 30, 30, 6)         60        
                                                                 
 conv2d_65 (Conv2D)          (None, 28, 28, 6)         330       
                                                                 
 average_pooling2d_34 (Avera  (None, 14, 14, 6)        0         
 gePooling2D)                                                    
                                                                 
 activation_34 (Activation)  (None, 14, 14, 6)         0         
                                                                 
 conv2d_66 (Conv2D)          (None, 12, 12, 16)        880       
                                                                 
 conv2d_67 (Conv2D)          (None, 10, 10, 16)        2320      
                                                     

In [105]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = model.fit(data_train_CNN, label_train, batch_size=64, epochs=40, validation_data=(data_test_CNN, label_test))

Epoch 1/40
938/938 [==============================] - 27s 28ms/step - loss: 0.2663 - accuracy: 0.9172 - val_loss: 0.0756 - val_accuracy: 0.9761
Epoch 2/40
938/938 [==============================] - 25s 27ms/step - loss: 0.0681 - accuracy: 0.9787 - val_loss: 0.0463 - val_accuracy: 0.9863
Epoch 3/40
938/938 [==============================] - 25s 27ms/step - loss: 0.0440 - accuracy: 0.9860 - val_loss: 0.0433 - val_accuracy: 0.9863
Epoch 4/40
938/938 [==============================] - 25s 27ms/step - loss: 0.0351 - accuracy: 0.9887 - val_loss: 0.0496 - val_accuracy: 0.9840
Epoch 5/40
938/938 [==============================] - 26s 28ms/step - loss: 0.0276 - accuracy: 0.9913 - val_loss: 0.0440 - val_accuracy: 0.9867
Epoch 6/40
938/938 [==============================] - 26s 28ms/step - loss: 0.0244 - accuracy: 0.9922 - val_loss: 0.0340 - val_accuracy: 0.9902
Epoch 7/40
938/938 [==============================] - 24s 26ms/step - loss: 0.0201 - accuracy: 0.9934 - val_loss: 0.0248 - val_accuracy:

In [106]:
model.evaluate(data_test_CNN, label_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0337 - accuracy: 0.9930


[0.033708035945892334, 0.9929999709129333]

MSE is not good for classification. Non-convex

In [15]:
model = models.Sequential()
model.add(layers.Conv2D(6, 3, activation='relu', input_shape=data_train_CNN.shape[1:]))
model.add(layers.Conv2D(6, 3, activation='relu', input_shape=data_train_CNN.shape[1:]))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(16, 3, activation='relu'))
model.add(layers.Conv2D(16, 3, activation='relu'))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(120, 3, activation='relu'))
model.add(layers.Conv2D(120, 3, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(84, activation='relu'))
model.add(layers.Dense(10, activation='sigmoid'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 6)         60        
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 6)         330       
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 activation (Activation)     (None, 14, 14, 6)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 16)        880       
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 16)        2320      
                                                        

In [16]:
model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
history = model.fit(data_train_CNN, label_train, batch_size=64, epochs=40, validation_data=(data_test_CNN, label_test))

Epoch 1/40
938/938 [==============================] - 26s 27ms/step - loss: 20.3208 - accuracy: 0.0989 - val_loss: 20.2422 - val_accuracy: 0.0980
Epoch 2/40
938/938 [==============================] - 25s 26ms/step - loss: 20.2774 - accuracy: 0.0987 - val_loss: 20.2422 - val_accuracy: 0.0980
Epoch 3/40
938/938 [==============================] - 25s 27ms/step - loss: 20.2774 - accuracy: 0.0987 - val_loss: 20.2422 - val_accuracy: 0.0980
Epoch 4/40
938/938 [==============================] - 26s 27ms/step - loss: 20.2774 - accuracy: 0.0987 - val_loss: 20.2422 - val_accuracy: 0.0980
Epoch 5/40
938/938 [==============================] - 27s 28ms/step - loss: 20.2774 - accuracy: 0.0987 - val_loss: 20.2422 - val_accuracy: 0.0980
Epoch 6/40
938/938 [==============================] - 29s 31ms/step - loss: 20.2774 - accuracy: 0.0987 - val_loss: 20.2422 - val_accuracy: 0.0980
Epoch 7/40
938/938 [==============================] - 29s 31ms/step - loss: 20.2774 - accuracy: 0.0987 - val_loss: 20.2422 -

In [17]:
model.evaluate(data_test_CNN, label_test)

313/313 [==============================] - 1s 2ms/step - loss: 20.2422 - accuracy: 0.0980


[20.24220085144043, 0.09799999743700027]

More tuning

In [12]:
model = models.Sequential()
model.add(layers.Conv2D(32,3, activation='relu', input_shape=data_train_CNN.shape[1:]))
model.add(layers.MaxPooling2D(2,strides=(2,2)))
model.add(layers.Conv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D(2,strides=(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128,input_shape=(3136,), activation=None))
model.add(layers.Dense(10, input_shape=(128,), activation=None))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 128)               2

In [13]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = model.fit(data_train_CNN, label_train, batch_size=64, epochs=40, validation_data=(data_test_CNN, label_test))

Epoch 1/40
938/938 [==============================] - 29s 31ms/step - loss: 2.4661 - accuracy: 0.2295 - val_loss: 2.3026 - val_accuracy: 0.1254
Epoch 2/40
938/938 [==============================] - 28s 29ms/step - loss: 2.3026 - accuracy: 0.1251 - val_loss: 2.3026 - val_accuracy: 0.1254
Epoch 3/40
938/938 [==============================] - 25s 27ms/step - loss: 2.3026 - accuracy: 0.1251 - val_loss: 2.3026 - val_accuracy: 0.1254
Epoch 4/40
938/938 [==============================] - 24s 26ms/step - loss: 2.3026 - accuracy: 0.1251 - val_loss: 2.3026 - val_accuracy: 0.1254
Epoch 5/40
938/938 [==============================] - 23s 25ms/step - loss: 2.3026 - accuracy: 0.1251 - val_loss: 2.3026 - val_accuracy: 0.1254
Epoch 6/40
682/938 [====================>.........] - ETA: 6s - loss: 2.3026 - accuracy: 0.1245

In [ ]:
model.evaluate(data_test_CNN, label_test)

313/313 [==============================] - 1s 2ms/step - loss: 20.2422 - accuracy: 0.0980


[20.24220085144043, 0.09799999743700027]